In [2]:
from pyrosm import get_data, OSM
import numpy as np
import pandas as pd
import geopandas as gpd
from queue import PriorityQueue

/home/sofus/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
class Graph:
    def __init__(self, vertices : list, edges : list, neighbours : dict, weights : dict, positions : dict):
        self.vertices = vertices
        self.edges = edges
        self.neighbours = neighbours
        self.weights = weights
        self.positions = positions

In [30]:
def Dijkstra(G : Graph, source, target):
    dist = {source: 0}
    prev = {}
    Q = PriorityQueue()
    for v in G.vertices:
        if v != source:
            dist[v] = np.inf
            prev[v] = np.inf
        
        Q.put((dist[v], v))
    
    print(type(Q.queue))
    
    while not Q.empty():
        w, u = Q.get()
        if u in G.neighbours.keys():
            for v in G.neighbours[u]:
                alt = dist[u] + G.weights[(u, v)]
                if alt < dist[v]:
                    Q.queue.remove((dist[v], v))
                    dist[v] = alt
                    prev[v] = u
                    Q.put((dist[v], v))
    
    return dist, prev

In [21]:
fp = get_data("copenhagen")
osm = OSM(fp)
raw_vertices, raw_edges = osm.get_network("driving", nodes=True)
vertices = []
positions = {}
for i, lon, lat in zip(raw_vertices["id"], raw_vertices["lon"], raw_vertices["lat"]):
    vertices.append(i)
    positions[i] = (lon, lat)

edges = []
neighbours = {}
weights = {}
for u, v, length in zip(raw_edges["u"], raw_edges["v"], raw_edges["length"]):
    edges.append((u, v))
    weights[(u, v)] = length

    if u in neighbours.keys():
        neighbours[u] += [v]
    else:
        neighbours[u] = [v]

In [31]:
source = 25981317
target = 687890
G = Graph(vertices, edges, neighbours, weights, positions)
dist, prev = Dijkstra(G, source, target)
prev

<class 'list'>


ValueError: list.remove(x): x not in list